In [167]:
import os
import torch
from pathlib import Path
!pip install xarray netCDF4
import xarray
import netCDF4
HOME = Path.home()
print("using home dir:", HOME)

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print("\nCuda is available: "+str(torch.cuda.is_available()))

!nvcc --version

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
using home dir: /home/ec2-user
Thu Jun  8 23:13:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   30C    P8    20W / 300W |      2MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+--------------------

In [168]:
# download data from s3
import boto3
import netCDF4
import os
import boto3
import sagemaker
import xarray as xr
import pandas as pd
from pathlib import Path
HOME=Path.home()
%alias mkdatadir mkdir -p %l
sess = sagemaker.Session()
bucket = sess.default_bucket()                    # Set a default S3 bucket
print(f"Using s3 bucket: {bucket}")
DATA_DIR=f"{HOME}/SageMaker/data_downloads"
MODEL_DATA=f"{HOME}/SageMaker/model_data"
print(f"Using download data path: {DATA_DIR}")
print(f"Using model data path: {DATA_DIR}")
%mkdatadir $DATA_DIR

Using s3 bucket: sagemaker-us-west-2-014690315417
Using download data path: /home/ec2-user/SageMaker/data_downloads
Using model data path: /home/ec2-user/SageMaker/data_downloads


In [169]:
# download the data from s3


def sync_s3_to_local(bucket_name, prefix, local_directory):
    s3 = boto3.client('s3')

    # List all objects in the bucket with the specified prefix
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    objects = response['Contents']

    # Sync each object to the local directory
    for obj in objects:
        key = obj['Key']
        local_path = os.path.join(local_directory, key)

        # Create the local directory if it doesn't exist
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        # Check if the local file already exists
        if os.path.exists(local_path):
            print(f"Skipping {key} (already exists)")
            continue

        # Download the object from S3 to the local directory
        s3.download_file(bucket_name, key, local_path)
        print(f"Downloaded {key}")

# Usage example
prefix = "data/era5_labeled_regridded"
sync_s3_to_local(bucket,prefix,  DATA_DIR)


Skipping data/era5_labeled_regridded/2018-01-01.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-02.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-03.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-04.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-05.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-06.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-07.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-08.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-09.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-10.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-11.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-12.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-13.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-14.nc (already exists)
Skipping data/era5_labeled_regridded/2018-01-15.

In [170]:
import glob
source_directory = '/home/ec2-user/SageMaker/data_downloads/data/era5_labeled_regridded'
train_directory = f'{MODEL_DATA}/train'
val_directory = f'{MODEL_DATA}/val'
test_directory = f'{MODEL_DATA}/test'

%mkdatadir $train_directory
%mkdatadir $val_directory
%mkdatadir $test_directory


val_pattern = '2018-12-*.nc'
test_pattern = '2018-11-*.nc'
print(MODEL_DATA, train_directory)
# Get the list of files in the source directory matching the pattern
# val and test should not overlap
val_files = glob.glob(os.path.join(source_directory, file_pattern))
test_files = glob.glob(os.path.join(source_directory, test_pattern))
train_files = [f for f in all_files if f not in val_files and f not in test_files]
# Create symbolic links for each file in the target directory

n_train, n_val, n_test = 1, 1, 1

train_files=train_files[0:n_train]
val_files=val_files[0:n_val]
test_files=test_files[0:n_test]


# Iterate over the files and remove symbolic links
print("unlinking existing files")
for directory in [train_directory, val_directory, test_directory]:
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.islink(file_path):
            os.unlink(file_path)

print("linking downloads to model data")
for file in train_files:
    source_path = file
    target_path = os.path.join(train_directory, os.path.basename(file))
    os.symlink(source_path, target_path)

for file in val_files:
    source_path = file
    target_path = os.path.join(val_directory, os.path.basename(file))
    os.symlink(source_path, target_path)

for file in test_files:
    source_path = file
    target_path = os.path.join(test_directory, os.path.basename(file))
    os.symlink(source_path, target_path)


print("done")
! echo Training files && ls $train_directory
print()
! echo Validation files && ls $val_directory
print()
! echo Test files && ls $test_directory

/home/ec2-user/SageMaker/model_data /home/ec2-user/SageMaker/model_data/train
unlinking existing files
linking downloads to model data
done
Training files
2018-01-01.nc

Validation files
2018-12-08.nc

Test files
2018-11-18.nc


In [175]:
%cd $HOME/SageMaker/ClimateNet
%pwd
JACCARD_PATH = "./models/Baseline"
%ls $JACCARD_PATH
%cat $JACCARD_PATH/config.json

/home/ec2-user/SageMaker/ClimateNet
config.json  history.csv  weights.pth
{
    "architecture": "cgnet",
    "seed": 42,
    "lr": 0.001,
    "train_batch_size": 4,
    "pred_batch_size": 8,
    "epochs": 15,
    "fields": {
        "TMQ": {
            "mean": 19.21859,
            "std": 15.81723
        },
        "U850": {
            "mean": 1.55302,
            "std": 8.29764
        },
        "V850": {
            "mean": 0.25413,
            "std": 6.23163
        },
        "PSL": {
            "mean": 100814.414,
            "std": 1461.2227
        }
    },
    "labels": [
        "Background",
        "Tropical Cyclone",
        "Atmospheric River"
    ],
    "loss": "jaccard",
    "scheduler": false,
    "description": "Baseline: CGNet with jaccard loss trained on original ClimateNet with no learning rate scheduling."
}

In [176]:
import train_eval 
model_path = f"{HOME}/SageMaker/ClimateNet/{JACCARD_PATH}/"
predictions = train_eval.run(model_path, MODEL_DATA)

# !python3 train_eval.py \
# -d $MODEL_DATA \
# -m $HOME/SageMaker/ClimateNet/$JACCARD_PATH/


================ Training epoch #1 (cuda:0) ================


Loss: 0.88028 (jaccard) | LR: 0.001:   0%|          | 0/1 [00:01<?, ?it/s]/home/ec2-user/SageMaker/ClimateNet/climatenet/models.py:123: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history = history.append({'minibatch_loss': train_loss.item()}, ignore_index=True)
Loss: 0.88028 (jaccard) | LR: 0.001: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:62: RuntimeWarning: invalid value encountered in divide
  recall = true_positives / (true_positives + false_negatives)
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:64: RuntimeWarning: invalid value encountered in divide
  sensitivity = true_positives / (true_positives + false_negatives)
/home/ec2-user/SageMaker/ClimateNet/climatenet/models.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea


Training loss: 0.88028 (jaccard) 
Classes:      [    BG         TCs        ARs   ]
IoUs:         [0.42702956 0.00070775 0.        ]  | Mean:  0.14257910595743686
Dice score:   [0.59848734 0.00141451 0.        ]  | Mean:  0.19996728233561287
Precision:    [9.98893993e-01 7.10408911e-04 0.00000000e+00]
Recall:       [0.42723157 0.15923567        nan]
Specificity:  [0.26509993 0.85582275 0.57161642]
Sensitivity:  [0.42723157 0.15923567        nan]
[[4.270e+01 1.441e+01 4.283e+01]
 [4.700e-02 1.000e-02 7.000e-03]
 [0.000e+00 0.000e+00 0.000e+00]]

---------- Validation (cuda:0) ----------


100%|██████████| 1/1 [00:00<00:00,  2.79it/s]
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:61: RuntimeWarning: invalid value encountered in divide
  precision = true_positives / (true_positives + false_positives)
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:62: RuntimeWarning: invalid value encountered in divide
  recall = true_positives / (true_positives + false_negatives)
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:64: RuntimeWarning: invalid value encountered in divide
  sensitivity = true_positives / (true_positives + false_negatives)
/home/ec2-user/SageMaker/ClimateNet/climatenet/models.py:161: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  history = history.append({'epoch_val_loss': val_loss, 'learning_rate': self.optimizer.param_groups[0]["lr"],\



Validation loss: 0.90516 (jaccard)
Classes:      [    BG         TCs        ARs   ]
IoUs:         [1.67978668e-01 3.92403769e-04 1.00000000e+00]  | Mean:  0.389457023932954
Dice score:   [2.87639959e-01 7.84499698e-04 1.00000000e+00]  | Mean:  0.4294748195301074
Precision:    [9.99985697e-01 3.92404902e-04            nan]
Recall:       [0.16797907 0.99269759        nan]
Specificity:  [0.26509993 0.85582275 0.57161642]
Sensitivity:  [0.42723157 0.15923567        nan]
[[1.679e+01 8.317e+01 0.000e+00]
 [0.000e+00 3.300e-02 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00]]

================ Training epoch #2 (cuda:0) ================


...


Validation loss: 0.85758 (jaccard)
Classes:      [    BG         TCs        ARs   ]
IoUs:         [0.77750171 0.         0.        ]  | Mean:  0.2591672355746046
Dice score:   [0.87482527 0.         0.        ]  | Mean:  0.29160842388420133
Precision:    [0.99957714 0.         0.        ]
Recall:       [0.77775752 0.                nan]
Specificity:  [0.         0.9500093  0.94264264]
Sensitivity:  [0.89261501 0.                nan]
[[7.775e+01 2.367e+00 1.985e+01]
 [3.300e-02 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00]]

---------- Evaluation (cuda:0) ----------


100%|██████████| 1/1 [00:00<00:00,  2.84it/s]
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:62: RuntimeWarning: invalid value encountered in divide
  recall = true_positives / (true_positives + false_negatives)
/home/ec2-user/SageMaker/ClimateNet/climatenet/utils/metrics.py:64: RuntimeWarning: invalid value encountered in divide
  sensitivity = true_positives / (true_positives + false_negatives)



Test loss: 1.71749 (jaccard)
Classes:      [    BG         TCs        ARs   ]
IoUs:         [0.77015531 0.         0.        ]  | Mean:  0.25671843540521316
Dice score:   [0.87015563 0.         0.        ]  | Mean:  0.29005187793837545
Precision:    [0.99796656 0.         0.        ]
Recall:       [0.77136577 0.                nan]
Specificity:  [0.         0.97164041 0.80003964]
Sensitivity:  [0.77136577 0.                nan]
[[77.016  2.832 19.996]
 [ 0.157  0.     0.   ]
 [ 0.     0.     0.   ]]


100%|██████████| 1/1 [00:00<00:00,  2.81it/s]

[<xarray.DataArray (time: 8, lat: 768, lon: 1152)>
array([[[2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        ...,
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2]],

       [[2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        ...,
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2]],

       [[2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        ...,
...
        ...,
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2]],

       [[2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        ...,
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2]],

       [[2, 2, 2, ..., 2, 2, 2],
        [2, 2, 2, ..., 2, 2, 2],
        [2, 2,

In [190]:
# predictions.to_netcdf("predictions.nc")
# print("done")
ds = predictions.to_dataset(name="predictions")
ds.to_netcdf("predictions.nc")


In [ ]:
# TODO run this w cartopy
# predictions.plot()
p = predictions.isel(time=0).plot(
    subplot_kws=dict(projection=ccrs.Orthographic(-80, 35), facecolor="gray"),
    transform=ccrs.PlateCarree(),
)
p.axes.coastlines()
p.axes.gridlines()
plt.draw()